In [85]:
import os
import csv
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import re
import seaborn as sns
import networkx as nx
from networkx.algorithms import community

In [181]:
# Extract full list of authors
f = open('all.txt', encoding='utf-8').read()
coauthor = []
n = 1
for item in f:
    start = f.find('AU ', n)
    end = f.find('AF ', start)
    if start != -1:
        pb = f.find('PT ',n)
        start = f.find('AU ',pb)
        end = f.find('AF ',start)
        result = f[start:end]
        ed = f.find('ER ', end)
        co = result.find('\n ',start)
        name = result[co+3:]
        coauthor.append(name)
        n = ed + 1
    else:
        break

In [182]:
coauthor = pd.DataFrame(data=coauthor)
coauthor.columns = ['First Author']
coauthor.head()

,First Author
0,"Ahlqvist, T\n"
1,"Ahlsen, B\n Bondevik, H\n Mengshoel, AM\n..."
2,"Ahmad, AA\n Alsaad, AM\n"
3,"Ahmad, A\n Senapati, S\n Khan, MI\n Kum..."
4,"Ahmad, B\n Gunduz, O\n Stoyanov, S\n Pe..."


In [183]:
da = coauthor['First Author'].str.split('\n', expand=True).stack()
da = da.reset_index(level=1, drop=True).rename('Coauthor')
fa = coauthor['First Author'].map(lambda x:x.rsplit('\n ')[0])
fa = fa.str.replace('\n', '')
fa = fa.str.replace('\n', '')
da = pd.DataFrame(data=da)
fa = pd.DataFrame(data=fa)
name = fa.join(da)
name = name[name['Coauthor'].map(len) > 0]
name.head()

,First Author,Coauthor
0,"Ahlqvist, T","Ahlqvist, T"
1,"Ahlsen, B","Ahlsen, B"
1,"Ahlsen, B","Bondevik, H"
1,"Ahlsen, B","Mengshoel, AM"
1,"Ahlsen, B","Solbraekke, KN"


In [184]:
data = pd.read_csv('data.csv')
df = data.join(name)
df.head()

,Publication Type,Authors,Title,Language,Document Type,Author Keywords,Author Country,Cited References,Cited Reference Count,CR count,...,Research Areas,UC 180,UC since 2013,Publisher,Publisher City,Publication Date,Year Published,Page Count,First Author,Coauthor
0,J,"Ahlqvist, T",From information society to biosociety? On so...,English,Article,information society; biosociety; transition; ...,Finland,"AHLQVIST T, 2003, KEYS FUTURES SOC REF",21,8,...,Business & Economics; Public Administration,1,6,ELSEVIER SCIENCE INC,NEW YORK,JUN,2005,19,"Ahlqvist, T","Ahlqvist, T"
1,J,"Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,12,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,"Ahlsen, B","Ahlsen, B"
1,J,"Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,12,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,"Ahlsen, B","Bondevik, H"
1,J,"Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,12,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,"Ahlsen, B","Mengshoel, AM"
1,J,"Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,12,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,"Ahlsen, B","Solbraekke, KN"


In [185]:
df.shape

(491999, 22)

In [186]:
df = df[['Publication Type','Coauthor', 'First Author','Title','Language','Document Type','Author Keywords','Author Country','Cited References','Cited Reference Count','CR count','WOS Cited count','WOS Categories','Research Areas','UC 180','UC since 2013','Publisher','Publisher City','Publication Date','Year Published','Page Count']]
# Remove error first author
df = df[df['First Author'].str.len()<20]
# Remove when first author is coauthor
df = df[df['Coauthor']!=df['First Author']]
df.head()

,Publication Type,Coauthor,First Author,Title,Language,Document Type,Author Keywords,Author Country,Cited References,Cited Reference Count,...,WOS Cited count,WOS Categories,Research Areas,UC 180,UC since 2013,Publisher,Publisher City,Publication Date,Year Published,Page Count
1,J,"Bondevik, H","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,12,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8
1,J,"Mengshoel, AM","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,12,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8
1,J,"Solbraekke, KN","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,12,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8
2,J,"Alsaad, AM","Ahmad, AA",Adhesive B-doped DLC films on biomedical allo...,English,Article,boron-doped DLC; biomedical materials; adhesi...,Jordan,"AHMAD AA, 1996, THESIS U NEBRASKALIN",31,...,5,"Materials Science, Multidisciplinary",Materials Science,1,27,INDIAN ACAD SCIENCES,BANGALORE,MAR,2007,8
3,J,"Senapati, S","Ahmad, A",Extra-/Intracellular Biosynthesis of Gold Nan...,English,Article,Biosynthesis; Fungus; Nanoparticles; Gold; En...,India,"Ahmad A, 2003, NANOTECHNOLOGY, V14, P824, DOI...",37,...,117,Nanoscience & Nanotechnology; Materials Scien...,Science & Technology - Other Topics; Material...,0,26,AMER SCIENTIFIC BLISHERS,VALENCIA,JUN 5,2005,7


In [187]:
df.shape

(377482, 21)

In [188]:
data = df

In [189]:
# Split Reserach Areas
r = data[['Title','Research Areas','Year Published']]
ra = r['Research Areas'].str.split(';').apply(Series, 1).stack()
ra.index = ra.index.droplevel(-1)
ra.name = 'Research Areas'
del r['Research Areas']
r = r.join(ra)

In [190]:
r_imp = r.groupby(['Research Areas','Year Published']).size()/r.groupby(['Year Published']).size()
r_imp = pd.DataFrame(data=r_imp)

In [191]:
r_imp['Areas Impact'] = r.groupby(['Research Areas','Year Published']).size()/r.groupby(['Year Published']).size()

In [192]:
r_imp.head()

0  Areas Impact
Research Areas Year Published                        
               1987            0.002198      0.002198
               1988            0.008428      0.008428
               1989            0.000284      0.000284
               1990            0.014919      0.014919
               1991            0.022215      0.022215

In [193]:
# Research Areas centrality
ra_impact = pd.merge(r, r_imp,  on=['Research Areas','Year Published'])
r_impact = ra_impact[['Research Areas','Title','Year Published','Areas Impact']]
r_impact = r_impact.drop_duplicates()
r_impact = r_impact.groupby(['Title','Year Published']).sum()

In [194]:
# Merge impact
data = pd.merge(data, r_impact, on=['Title','Year Published'],how='left')

In [195]:
data.head()

,Publication Type,Coauthor,First Author,Title,Language,Document Type,Author Keywords,Author Country,Cited References,Cited Reference Count,...,WOS Categories,Research Areas,UC 180,UC since 2013,Publisher,Publisher City,Publication Date,Year Published,Page Count,Areas Impact
0,J,"Bondevik, H","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820
1,J,"Mengshoel, AM","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820
2,J,"Solbraekke, KN","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820
3,J,"Alsaad, AM","Ahmad, AA",Adhesive B-doped DLC films on biomedical allo...,English,Article,boron-doped DLC; biomedical materials; adhesi...,Jordan,"AHMAD AA, 1996, THESIS U NEBRASKALIN",31,...,"Materials Science, Multidisciplinary",Materials Science,1,27,INDIAN ACAD SCIENCES,BANGALORE,MAR,2007,8,0.086433
4,J,"Senapati, S","Ahmad, A",Extra-/Intracellular Biosynthesis of Gold Nan...,English,Article,Biosynthesis; Fungus; Nanoparticles; Gold; En...,India,"Ahmad A, 2003, NANOTECHNOLOGY, V14, P824, DOI...",37,...,Nanoscience & Nanotechnology; Materials Scien...,Science & Technology - Other Topics; Material...,0,26,AMER SCIENTIFIC BLISHERS,VALENCIA,JUN 5,2005,7,0.301268


In [196]:
data.shape

(377482, 22)

In [197]:
# Split Author Keywords
a = data[['Title','Author Keywords','Year Published']]
ak = a['Author Keywords'].str.split(';').apply(Series, 1).stack()
ak.index = ak.index.droplevel(-1)
ak.name = 'Author Keywords'
del a['Author Keywords']
a = a.join(ak)

In [198]:
a_imp = a.groupby(['Author Keywords','Year Published']).size()/a.groupby(['Year Published']).size()
a_imp = pd.DataFrame(data = a_imp)
a_imp['Keywords Impact'] = a.groupby(['Author Keywords','Year Published']).size()/a.groupby(['Year Published']).size()

In [199]:
a_imp.head()

0  Keywords Impact
Author Keywords Year Published                           
                1986            0.111313         0.111313
                1987            0.065465         0.065465
                1988            0.080690         0.080690
                1989            0.063190         0.063190
                1990            0.090009         0.090009

In [200]:
ak_impact = pd.merge(a, a_imp,  on=['Author Keywords','Year Published'])
a_impact = ak_impact[['Title','Year Published','Keywords Impact']]
a_impact = a_impact.drop_duplicates()
a_impact = a_impact.groupby(['Title','Year Published']).sum()
# Merge impact
data = pd.merge(data, a_impact, on=['Title','Year Published'],how='left')

In [201]:
data.head()

,Publication Type,Coauthor,First Author,Title,Language,Document Type,Author Keywords,Author Country,Cited References,Cited Reference Count,...,Research Areas,UC 180,UC since 2013,Publisher,Publisher City,Publication Date,Year Published,Page Count,Areas Impact,Keywords Impact
0,J,"Bondevik, H","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481
1,J,"Mengshoel, AM","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481
2,J,"Solbraekke, KN","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,Rehabilitation,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481
3,J,"Alsaad, AM","Ahmad, AA",Adhesive B-doped DLC films on biomedical allo...,English,Article,boron-doped DLC; biomedical materials; adhesi...,Jordan,"AHMAD AA, 1996, THESIS U NEBRASKALIN",31,...,Materials Science,1,27,INDIAN ACAD SCIENCES,BANGALORE,MAR,2007,8,0.086433,0.003063
4,J,"Senapati, S","Ahmad, A",Extra-/Intracellular Biosynthesis of Gold Nan...,English,Article,Biosynthesis; Fungus; Nanoparticles; Gold; En...,India,"Ahmad A, 2003, NANOTECHNOLOGY, V14, P824, DOI...",37,...,Science & Technology - Other Topics; Material...,0,26,AMER SCIENTIFIC BLISHERS,VALENCIA,JUN 5,2005,7,0.301268,0.002763


In [202]:
data.shape

(377482, 23)

In [203]:
#Calculate every coauthor's current career length in every year
c = data[['Coauthor','Year Published']]
c_g = c.groupby('Coauthor').min()
c_g = c_g.rename(columns={'Year Published': 'Start Year'})
cl = pd.merge(c, c_g, on=['Coauthor'])
cl['Career Length'] = cl['Year Published']-cl['Start Year']
cl = cl[['Coauthor','Year Published','Career Length']]
cl.head()

,Coauthor,Year Published,Career Length
0,"Bondevik, H",2014,10
1,"Bondevik, H",2004,0
2,"Mengshoel, AM",2014,10
3,"Mengshoel, AM",2004,0
4,"Solbraekke, KN",2014,10


In [204]:
cl.shape

(377482, 3)

In [205]:
#Merge 
data = data.merge(cl.drop_duplicates(subset=['Coauthor','Year Published']), how='left')

In [206]:
data.head()

,Publication Type,Coauthor,First Author,Title,Language,Document Type,Author Keywords,Author Country,Cited References,Cited Reference Count,...,UC 180,UC since 2013,Publisher,Publisher City,Publication Date,Year Published,Page Count,Areas Impact,Keywords Impact,Career Length
0,J,"Bondevik, H","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481,10
1,J,"Mengshoel, AM","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481,10
2,J,"Solbraekke, KN","Ahlsen, B",(Un)doing gender in a rehabilitation context:...,English,Article,Femininity; gender; masculinity; muscles; pain,Norway,"Ahlsen B, 2012, DISABIL REHABIL, V34, P1765, ...",53,...,0,10,INFORMA HEALTHCARE,LONDON,AUG,2014,8,0.203820,0.077481,10
3,J,"Alsaad, AM","Ahmad, AA",Adhesive B-doped DLC films on biomedical allo...,English,Article,boron-doped DLC; biomedical materials; adhesi...,Jordan,"AHMAD AA, 1996, THESIS U NEBRASKALIN",31,...,1,27,INDIAN ACAD SCIENCES,BANGALORE,MAR,2007,8,0.086433,0.003063,0
4,J,"Senapati, S","Ahmad, A",Extra-/Intracellular Biosynthesis of Gold Nan...,English,Article,Biosynthesis; Fungus; Nanoparticles; Gold; En...,India,"Ahmad A, 2003, NANOTECHNOLOGY, V14, P824, DOI...",37,...,0,26,AMER SCIENTIFIC BLISHERS,VALENCIA,JUN 5,2005,7,0.301268,0.002763,6


In [234]:
data.shape

(377482, 24)

In [237]:
data.to_csv('data_v2.csv', encoding='utf-8', index=False)

In [240]:
# Use pagerank algorithm to find important authors
df=data[(data['Year Published']<=2000)]
G=nx.DiGraph()   
G.add_nodes_from(df.Coauthor)
G.nodes()
G.add_node('None')
subset = df[['First Author','Coauthor']]
tuples = [tuple(x) for x in subset.values] 
G.add_edges_from(tuples)
G.number_of_edges()
pg = nx.pagerank(G)
rank = pd.DataFrame.from_dict(pg,orient='index')
rank['Author Rank'] = pd.DataFrame.from_dict(pg,orient='index')
rank['Coauthor'] = rank.index
rank = rank[['Author Rank','Coauthor']]
df1=data[(data['Year Published']==2000)]
page=df1.merge(rank.drop_duplicates(subset=['Coauthor']), how='left')

In [241]:
page.shape

(4316, 25)

In [242]:
n = 2001
for item in data:
    if n != 2018:
        df=data[(data['Year Published']<=n)]
        G=nx.DiGraph()   
        G.add_nodes_from(df.Coauthor)
        G.nodes()
        G.add_node('None')
        subset = df[['First Author','Coauthor']]
        tuples = [tuple(x) for x in subset.values] 
        G.add_edges_from(tuples)
        G.number_of_edges()
        pg = nx.pagerank(G)
        rank = pd.DataFrame.from_dict(pg,orient='index')
        rank['Author Rank'] = pd.DataFrame.from_dict(pg,orient='index')
        rank['Coauthor'] = rank.index
        rank = rank[['Author Rank','Coauthor']]
        df1=data[(data['Year Published']==n)]
        pr=df1.merge(rank.drop_duplicates(subset=['Coauthor']), how='left')
        n = n + 1
        page = pd.concat([page,pr])
    else:
        break

In [243]:
page.shape

(353496, 25)

In [244]:
#Use pagerank algorithm to find important publisher
df=data[(data['Year Published']<=2000)]
G=nx.DiGraph()   
G.add_nodes_from(df.Coauthor)
G.nodes()
G.add_node('None')
subset = df[['Publisher','Coauthor']]
tuples = [tuple(x) for x in subset.values] 
G.add_edges_from(tuples)
G.number_of_edges()
pg = nx.pagerank(G)
rank = pd.DataFrame.from_dict(pg,orient='index')
rank['Publisher Rank'] = pd.DataFrame.from_dict(pg,orient='index')
rank['Coauthor'] = rank.index
rank = rank[['Publisher Rank','Coauthor']]
df2=data[(data['Year Published']==2000)]
pub=df2.merge(rank.drop_duplicates(subset=['Coauthor']), how='left')

In [246]:
pub.shape

(4316, 25)

In [247]:
n = 2001
for item in data:
    if n != 2018:
        df=data[(data['Year Published']<=n)]
        G=nx.DiGraph()   
        G.add_nodes_from(df.Coauthor)
        G.nodes()
        G.add_node('None')
        subset = df[['Publisher','Coauthor']]
        tuples = [tuple(x) for x in subset.values] 
        G.add_edges_from(tuples)
        G.number_of_edges()
        pg = nx.pagerank(G)
        rank = pd.DataFrame.from_dict(pg,orient='index')
        rank['Publisher Rank'] = pd.DataFrame.from_dict(pg,orient='index')
        rank['Coauthor'] = rank.index
        rank = rank[['Publisher Rank','Coauthor']]
        df2=data[(data['Year Published']==n)]
        pr=df2.merge(rank.drop_duplicates(subset=['Coauthor']), how='left')
        n = n + 1
        pub = pd.concat([pub,pr])
    else:
        break

In [248]:
pub.shape

(353496, 25)

In [250]:
pub_rank=pub[['Publisher','Year Published','Publisher Rank']]

In [258]:
data=page.merge(pub_rank.drop_duplicates(subset=['Publisher','Year Published']), how='left')

In [261]:
data.shape

(353496, 26)

In [259]:
data.to_csv('data_v3.csv', encoding='utf-8', index=False)